In [1]:
import random
import scipy.io as sio
import scipy
import operator
import itertools
import collections
from tabulate import tabulate
import numpy as np

N         = 10  # Number of jobs in the queue
P         = 2*N # Number of processors,special case for infinite proc
kk        = 2   # Pairs of applications
qos       = 0.9 # QoS threshold

# loading data
def loaddata(k):
    mat_contents = scipy.io.loadmat('../tmp/dat-multi-'+str(k)+'.mat')
    oct_struct   = mat_contents['dat']
    X ,Y         = oct_struct['xx'], oct_struct['yy']
    X, Y         = X[0,0], Y[0,0]
    return X, Y
    
with open('../data/All_app.tsv') as f:
    app_name = f.read().split('\n')

X,Y       = loaddata(2)
n         = len(app_name)

In [33]:
job_slice = collections.namedtuple('job_slice',\
            ['job_id','name','app_id','type','work','qos','deadline'])

# generate a random job queue
class job_queue:
    def __init__(self):
        self._job_queue = []
    def random(self, N, qos, app_name):
        """
        :type N:          int: Number of jobs
        :type qos:        float: qos constraint
        :type app_name:   list[str]: application names
        :rtype: job_queue:list[job_slice]: Set of jobs
        """    
        # Harcoding qos job names
        app_name_qos   = ['x264','vips','swish','swaptions','streamcluster',\
                      'STREAM','fluidanimate','blackscholes']
        num_qos   = 0
        job_queue = []
        for i in range(N):
            j_work     = random.choice([100,500,1000])
            if num_qos < P/2:
                j_type     = random.choice(['qos', 'batch'])
            else:
                j_type     = 'batch'

            if j_type      == 'qos':
                  
                j_qos      = qos
                j_deadline = (1+(1-qos))*j_work
                #j_deadline = None
                num_qos    += 1
                indices    = [i for i,item in enumerate(app_name) \
                              if item in app_name_qos]
                j_iden     = random.choice(indices)
                j_name     = app_name[j_iden-1]
            elif j_type    == 'batch':
                j_deadline = random.choice(range(j_work,3000,100))
                j_qos      = None
                indices    = [i for i,item in enumerate(app_name) \
                              if item not in app_name_qos] 
                j_iden     = random.choice(indices)
                j_name     = app_name[j_iden-1]
            job = job_slice(i, j_name, j_iden, j_type,  j_work, j_qos, j_deadline)
            job_queue.append(job);
        #job_queue.sort(key=operator.attrgetter('job'));
        self._job_queue = job_queue
        
    def __len__(self): 
        return len(self._job_queue)
    
    def __getitem__(self, position): 
        return self._job_queue[position]
    
    def append(self, item): 
        self._job_queue.append(item)
    
    def __repr__(self):
        return tabulate(list([job for job in self._job_queue] ), headers = job_slice._fields)
    


def nchoosek_rep(array, k):
    return list(itertools.combinations_with_replacement(array, k))

# class servers:
#     def __init__(self, Performance):
#         self.machines = []
#     # This function adds a job to the current job_queue on machine
#     # and returns true if the job could be successfully added
    
#     # This function adds a new machine
#     def update(self, machine):
#         return True

In [44]:
#description

schedule = collections.namedtuple('schedule',['interval','jobs',\
                                              'speeds','est_speeds'])  
# scheduling jobs
class machine:
    def __init__(self):
        self.utilization   = 0
        
        self._performance  = []
        self._schedules    = [] # list of schedules
        self._job_queue    = job_queue() # jobs on the machine 
        
    def fit(self, performance, app_name):
        self._performance = performance
        n         = len(app_name)
    def update(self, job):
        job_queue = self._job_queue
        job_queue.append(job)
        jobs = sorted(job_queue, key=operator.attrgetter('app_id'))

        app_id   = list([job.app_id for job in jobs])
        W        = list([job.work for job in jobs])
        deadline = list([job.deadline for job in jobs])
        #print(app_id,W,deadline)
        res = self.__lin_prog(app_id, W, deadline, Y, n)
        return res
    def __lin_prog(self, app_id, W, deadline, Y, n):
        
        a = nchoosek_rep(app_id, 2)
        aa = nchoosek_rep(range(len(app_id)), 2)
        b  = nchoosek_rep(range(n), 2)
        A  = np.zeros(shape=(len(app_id),len(a)))

        for i in range(len(a)):
            tmp = b.index(a[i])
            A[aa[i][0],i], A[aa[i][1],i] = Y[tmp, 0], Y[tmp,1]  

        A = np.concatenate([np.eye(len(app_id)), A.T])
        A = A.T
#         print('A', A.shape)
#         print('b', A.shape[1])
#         print('c', len(W))
        res = scipy.optimize.linprog([1]*A.shape[1], A_eq=A, b_eq=W)
                                    #options={"disp": True})
        #np.set_printoptions(precision=2);print(res.fun)
        return res


In [45]:
jobs = job_queue()
jobs.random(N, qos, app_name)

mac = machine()
mac.fit(Y,app_name)
A  = np.zeros(shape=(3,4))
type(A.shape[1])

for job in jobs:
    res = mac.update(job)
    print(res.fun)
print(jobs)


1000.0
999.409961473
1026.05854494
2026.05854494
2023.1083523
2078.09011725
2096.93177321
2136.64938315
3136.64938315
3272.27248121
  job_id  name              app_id  type      work    qos    deadline
--------  --------------  --------  ------  ------  -----  ----------
       0  streamcluster         22  batch     1000               1800
       1  dijkstra               8  qos        100    0.9         110
       2  jacobi                13  batch      500               2500
       3  svm_rfe               23  qos       1000    0.9        1100
       4  dijkstra               8  qos        500    0.9         550
       5  hotspot               12  batch      100               1000
       6  vips                  27  qos        100    0.9         110
       7  STREAM                21  qos        100    0.9         110
       8  particlefilter        19  batch     1000               1900
       9  vips                  27  qos        500    0.9         550


In [38]:
print(res)

None
